In [ ]:
import pandas as pd
import numpy as np
import requests
import csv

In [ ]:
LHEproducer = """import FWCore.ParameterSet.Config as cms

externalLHEProducer = cms.EDProducer("ExternalLHEProducer",
    args = cms.vstring('__GRIDPACK__'),
    nEvents = cms.untracked.uint32(5000),
    numberOfParameters = cms.uint32(1),
    outputFile = cms.string('cmsgrid_final.lhe'),
    generateConcurrently = cms.untracked.bool(False),
    scriptName = cms.FileInPath('GeneratorInterface/LHEInterface/data/run_generic_tarball_cvmfs.sh')
)

#Link to datacards:
#https://github.com/cms-sw/genproductions/tree/master/bin/MadGraph5_aMCatNLO/cards/production/13p6TeV/HHresonant/Spin-2
"""

pythiaFragment = """from Configuration.Generator.Pythia8CommonSettings_cfi import *
from Configuration.Generator.MCTunesRun3ECM13p6TeV.PythiaCP5Settings_cfi import *
from Configuration.Generator.PSweightsPythia.PythiaPSweightsSettings_cfi import *

generator = cms.EDFilter("Pythia8ConcurrentHadronizerFilter",
    maxEventsToPrint = cms.untracked.int32(1),
    pythiaPylistVerbosity = cms.untracked.int32(1),
    filterEfficiency = cms.untracked.double(1.0),
    pythiaHepMCVerbosity = cms.untracked.bool(False),
    comEnergy = cms.double(13600.),
    PythiaParameters = cms.PSet(
        pythia8CommonSettingsBlock,
        pythia8CP5SettingsBlock,
        pythia8PSweightsSettingsBlock,
        processParameters = cms.vstring(
                                    '25:onMode = off',
                                    '25:onIfMatch = 5 -5'
          ),
        parameterSets = cms.vstring('pythia8CommonSettings',
                                    'pythia8CP5Settings',
                                    'pythia8PSweightsSettings',
                                    'processParameters')
    )
)"""

In [ ]:
print(LHEproducer+"\n"+pythiaFragment)

In [ ]:
years = ["2022","2022EE"]
masses = ["250", "260", "270", "280", "300", "350", "450", "550", "600", "650", "700", "800", "1000",
"1200", "1400", "1600", "1800", "2000", "2500", "3000", "4000", "5000"]

events = [400000]*13+[100000]*9
for year in years:
    with open('BulkGraviton_HH_4B_'+ year + '.csv', 'w') as csvfile:
        csvwriter = csv.writer(csvfile, delimiter=',',
                            quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csvwriter.writerow(['Dataset name','Events', 'fragment','notes','Generator', 'mcdbid','time','size'])

        for mass, event in zip(masses, events):
            version="2.9.13"
            generators="Madgraph_" + version + "  Pythia8"
            gp = "/cvmfs/cms.cern.ch/phys_generator/gridpacks/RunIII/13p6TeV/slc7_amd64_gcc10/MadGraph5_aMCatNLO/GF_HH_Spin2/BulkGraviton_hh_GF_HH_narrow_M{0}_slc7_amd64_gcc10_CMSSW_12_4_8_tarball.tar.xz".format(mass)
            dataset_name = "GluGlutoBulkGravitontoHHto4B_M-{0}_narrow_TuneCP5_13p6TeV_madgraph-pythia8".format(mass)
            final_fragment = LHEproducer.replace('__GRIDPACK__',gp) + '\n\n' + pythiaFragment
            if year == "2022":
                tot_events = round(event * 0.22)
            elif year == "2022EE":
                tot_events = round(event * 0.78)
            note = "GluGlu BulkGraviton production m_BG = {0} GeV decaying into HH (m_H = 125 GeV) in the 4b final state (RunIII)".format(mass)
            mcdb_id = '0'
            time = '0.1'
            size = '30'
            csvwriter.writerow([dataset_name, tot_events, final_fragment, note, generators, mcdb_id, time, size])
        